In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext Cython
%timeit

In [2]:
import skimage
import numpy as np

In [3]:
im = skimage.io.imread('../images/pantalones.jpg')
im.shape

(1585, 672, 3)

In [4]:


def computeWhiteSpaceRatio(im):
    img_array = np.array(im)
    white_pixels = (img_array == (255,255,255)).all(axis=-1).sum()
    total_size = img_array.shape[0] * img_array.shape[1] 
    ratio = white_pixels / total_size
    return ratio


In [5]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef fastcomputeWhiteSpaceRatio(np.ndarray[np.uint8_t, ndim=3] img_array):
    cdef int height = img_array.shape[0]
    cdef int width = img_array.shape[1]
    cdef int x, y
    cdef long white_pixels = 0
    cdef long total_size = height * width
    cdef float ratio

    for y in range(height):
        for x in range(width):
            if img_array[y, x, 0] == 255 and img_array[y, x, 1] == 255 and img_array[y, x, 2] == 255:
                white_pixels += 1

    ratio = white_pixels / float(total_size)
    return ratio


Content of stderr:
In file included from /Users/dbuchaca/.cache/ipython/cython/_cython_magic_8166f43324b0af3388240fdf0bb70c20317f9c21.c:1220:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
/Users/dbuchaca/.cache/ipython/cython/_cython_magic_8166f43324b0af3388240fdf0bb70c20317f9c21.c:6751:26: warning: code will never be executed [-Wunreachable-code]
 

In [6]:
computeWhiteSpaceRatio(im)

0.31954709328526365

In [7]:
fastcomputeWhiteSpaceRatio(im)

0.3195470869541168

In [8]:
py_version = %timeit -o computeWhiteSpaceRatio(im)

29.6 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
cy_version = %timeit -o fastcomputeWhiteSpaceRatio(im)

539 µs ± 21.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
speedup = round(py_version.average / cy_version.average,2)

In [12]:
speedup

54.94

In [28]:
%%cython -a -c=-ffast-math -c=-funsafe-math-optimizations

cimport numpy as cnp
from cython cimport cdivision, boundscheck, wraparound

@boundscheck(False)
@wraparound(False)
cpdef double fastcomputeWhiteSpaceRatio2(cnp.uint8_t[:,:,:] img_array):
    """
    computeWhiteSpaceRatio computes the ratio between white and other colors in an image

    :return float - ratio in the [0, 1] range
    """
    cdef:
        double ratio
        int I = img_array.shape[0]
        int J = img_array.shape[1] 
        int C = img_array.shape[2]
        int counter_white_pixels = 0 

    for i in range(I):
         for j in range(J):
            if img_array[i,j,0] == 255 and img_array[i,j,1] == 255 and img_array[i,j,2] == 255:
                 counter_white_pixels +=1
                     
    total_size =  I * J 
    ratio = counter_white_pixels / total_size
    return ratio

Content of stderr:
In file included from /Users/dbuchaca/.cache/ipython/cython/_cython_magic_2a727f80952198091b528f670866edb68914fd83.c:1230:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
/Users/dbuchaca/.cache/ipython/cython/_cython_magic_2a727f80952198091b528f670866edb68914fd83.c:23906:26: warning: code will never be executed [-Wunreachable-code]


In [29]:
fastcomputeWhiteSpaceRatio2(im)

0.31954709328526365

In [30]:
cy_version2 = %timeit -o fastcomputeWhiteSpaceRatio2(im)

519 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
speedup2 = round(py_version.average / cy_version2.average,2)
speedup2

57.09

### Other alternatives

Note that making changing the inner loop to be a sum we don't win much

In [54]:
%%cython --annotate
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef fastcomputeWhiteSpaceRatio2(np.ndarray[np.uint8_t, ndim=3] img_array):
    cdef int height = img_array.shape[0]
    cdef int width = img_array.shape[1]
    cdef int x, y
    cdef long white_pixels = 0
    cdef long total_size = height * width
    cdef float ratio

    for y in range(height):
        for x in range(width):
            if img_array[y, x, 0] + img_array[y, x, 1] + img_array[y, x, 2] == 765:
                white_pixels += 1

    ratio = white_pixels / float(total_size)
    return ratio

Content of stderr:
In file included from /Users/dbuchaca/.cache/ipython/cython/_cython_magic_33fee714d853c0daeb29613cfe011c24926bed58.c:1224:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/Users/dbuchaca/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
/Users/dbuchaca/.cache/ipython/cython/_cython_magic_33fee714d853c0daeb29613cfe011c24926bed58.c:6747:26: warning: code will never be executed [-Wunreachable-code]
 

In [55]:
cy_version2 = %timeit -o fastcomputeWhiteSpaceRatio2(im)

544 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
speedup = round(py_version.average / cy_version2.average,2)
speedup

56.81